# Task-specific knowledge distillation for BERT using Hugging Face Transformers
### Text Classification Example using `BERT-Base` as Teacher and `BERT-Tiny` as Student

Welcome to our end-to-end task-specific knowledge distilattion Text-Classification example using Transformers, PyTorch & Amazon SageMaker. Distillation is the process of training a small "student" to mimic a larger "teacher". In this example, we will use [BERT-base](https://huggingface.co/textattack/bert-base-uncased-SST-2) as Teacher and [BERT-Tiny](https://huggingface.co/google/bert_uncased_L-2_H-128_A-2) as Student. We will use [Text-Classification](https://huggingface.co/tasks/text-classification) as task-specific knowledge distillation task and the [Stanford Sentiment Treebank v2 (SST-2)](https://paperswithcode.com/dataset/sst) dataset for training.


They are two different types of knowledge distillation, the Task-agnostic knowledge distillation (right) and the Task-specific knowledge distillation (left). In this example we are going to use the Task-specific knowledge distillation.

![knowledge-distillation](./imgs/knowledge-distillation.png)
_Task-specific distillation (left) versus task-agnostic distillation (right). Figure from FastFormers by Y. Kim and H. Awadalla [arXiv:2010.13382]._


In Task-specific knowledge distillation a "second step of distillation" is used to "fine-tune" the model on a given dataset. This idea comes from the [DistilBERT paper](https://arxiv.org/pdf/1910.01108.pdf) where it was shown that a student performed better than simply finetuning the distilled language model:

> We also studied whether we could add another step of distillation during the adaptation phase by fine-tuning DistilBERT on SQuAD using a BERT model previously fine-tuned on SQuAD as a teacher for an additional term in the loss (knowledge distillation). In this setting, there are thus two successive steps of distillation, one during the pre-training phase and one during the adaptation phase. In this case, we were able to reach interesting performances given the size of the model:79.8 F1 and 70.4 EM, i.e. within 3 points of the full model.

If you are more interested in those topics you should defintely read:
* [DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter](https://arxiv.org/abs/1910.01108)
* [FastFormers: Highly Efficient Transformer Models for Natural Language Understanding](https://arxiv.org/abs/2010.13382)

Especially the [FastFormers paper](https://arxiv.org/abs/2010.13382) contains great research on what works and doesn't work when using knowledge distillation.

---

Huge thanks to [Lewis Tunstall](https://www.linkedin.com/in/lewis-tunstall/) and his great [Weeknotes: Distilling distilled transformers](https://lewtun.github.io/blog/weeknotes/nlp/huggingface/transformers/2021/01/17/wknotes-distillation-and-generation.html#fn-1)


## Installation

In [1]:
#%pip install "pytorch==1.10.1"
%pip install transformers datasets tensorboard --upgrade

!sudo apt-get install git-lfs

Note: you may need to restart the kernel to use updated packages.


'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
%pip install accelerate==0.15.0 -U
%pip install transformers[torch]==4.28.1
%pip install prettytable

Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.28.1-py3-none-any.whl.metadata (109 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
Using cached tokenizers-0.13.3-cp311-cp311-win_amd64.whl (3.5 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


This example will use the [Hugging Face Hub](https://huggingface.co/models) as remote model versioning service. To be able to push our model to the Hub, you need to register on the [Hugging Face](https://huggingface.co/join).
If you already have an account you can skip this step.
After you have an account, we will use the `notebook_login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk.

In [1]:
from huggingface_hub import notebook_login

notebook_login()


## Setup & Configuration

In this step we will define global configurations and paramters, which are used across the whole end-to-end fine-tuning proccess, e.g. `teacher` and `studen` we will use.

In this example, we will use [BERT-base](textattack/bert-base-uncased-SST-2) as Teacher and [BERT-Tiny](https://huggingface.co/google/bert_uncased_L-2_H-128_A-2) as Student. Our Teacher is already fine-tuned on our dataset, which makes it easy for us to directly start the distillation training job rather than fine-tuning the teacher first to then distill it afterwards.

_**IMPORTANT**: This example will only work with a `Teacher` & `Student` combination where the Tokenizer is creating the same output._

Additionally, describes the [FastFormers: Highly Efficient Transformer Models for Natural Language Understanding](https://arxiv.org/abs/2010.13382) paper an additional phenomenon.
> In our experiments, we have observed that dis-
tilled models do not work well when distilled to a
different model type. Therefore, we restricted our
setup to avoid distilling RoBERTa model to BERT
or vice versa. The major difference between the
two model groups is the input token (sub-word) em-
bedding. We think that different input embedding
spaces result in different output embedding spaces,
and knowledge transfer with different spaces does
not work well

In [2]:
import collections
from typing import Union, List
import numpy as np
from tqdm.auto import tqdm
from transformers.trainer_utils import PredictionOutput
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers import TrainingArguments, Trainer, EvalPrediction, default_data_collator
from datasets import load_metric
student_id = "google/electra-small-discriminator"
teacher_id = "ahotrod/electra_large_discriminator_squad2_512"

# name for our repository on the hub
repo_name = "electra-distilled-qa"

Below are some checks to make sure the `Teacher` & `Student` are creating the same output.

In [3]:
from transformers import AutoTokenizer

# init tokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_id)
student_tokenizer = AutoTokenizer.from_pretrained(student_id)

# sample input
sample = "This is a basic example, with different words to test."

# assert results
assert teacher_tokenizer(sample) == student_tokenizer(sample), "Tokenizers haven't created the same output"


## Dataset & Pre-processing

https://huggingface.co/learn/nlp-course/en/chapter7/7

In [4]:
def prepare_train_features(examples: Union[str, List[str], List[List[str]]], tokenizer: PreTrainedTokenizer, 
                           pad_on_right: bool, max_length: int=512, doc_stride: int=128):
    "Tokenize and encode training examples in the SQuAD format"
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # label impossible answers with the index of the CLS token
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    "Tokenize and encode validation examples in the SQuAD format"
    tokenized_examples = tokenizer(
        examples['question' if pad_on_right else 'context'],
        examples['context' if pad_on_right else 'question'],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def convert_examples_to_features(dataset, tokenizer, num_train_examples, num_eval_examples, 
                                 max_length=512, doc_stride=128, seed=42):
    "Tokenize and encode the training and validation examples in the SQuAD format"
    max_length = max_length 
    doc_stride = doc_stride 
    pad_on_right = tokenizer.padding_side == "right"
    fn_kwargs = {
        "tokenizer": tokenizer,
        "max_length": max_length,
        "doc_stride": doc_stride,
        "pad_on_right": pad_on_right
    }
    train_enc = (dataset['train']
                 .shuffle(seed=seed)
                 .select(range(num_train_examples))
                 .map(prepare_train_features, fn_kwargs=fn_kwargs, batched=True, remove_columns=dataset["train"].column_names)
                )
    eval_enc = (dataset['validation']
                .shuffle(seed=seed)
                .select(range(num_eval_examples))
                .map(prepare_validation_features, fn_kwargs=fn_kwargs, batched=True, remove_columns=dataset["validation"].column_names)
               )
    eval_examples = dataset['validation'].shuffle(seed=seed).select(range(num_eval_examples))

    return train_enc, eval_enc, eval_examples

In [5]:
dataset_id="squad_v2"
dataset_config="sst2"

To load the `sst2` dataset, we use the `load_dataset()` method from the 🤗 Datasets library.


In [6]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

### Pre-processing & Tokenization

To distill our model we need to convert our "Natural Language" to token IDs. This is done by a 🤗 Transformers Tokenizer which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary). If you are not sure what this means check out [chapter 6](https://huggingface.co/course/chapter6/1?fw=tf) of the Hugging Face Course.

We are going to use the tokenizer of the `Teacher`, but since both are creating same output you could also go with the `Student` tokenizer.


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(teacher_id)

Additionally we add the `truncation=True` and `max_length=512` to align the length and truncate texts that are bigger than the maximum size allowed by the model.

In [10]:
num_train_examples = 3200
num_eval_examples = 320
train_ds, eval_ds, eval_examples = convert_examples_to_features(dataset, tokenizer, num_train_examples, num_eval_examples)
assert eval_examples.num_rows == num_eval_examples

In [11]:
metric = load_metric("squad_v2")

def squad_metrics(p: EvalPrediction):
    "Compute the Exact Match and F1-score metrics on SQuAD"
    return metric.compute(predictions=p.predictions, references=p.label_ids)

C:\Users\Kasra Sohrab\AppData\Local\Temp\ipykernel_7080\936429260.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
c:\Users\Kasra Sohrab\.conda\envs\nlp\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


## Distilling the model using `PyTorch` and `DistillationTrainer`


Now that our `dataset` is processed, we can distill it. Normally, when fine-tuning a transformer model using PyTorch you should go with the `Trainer-API`. The [Trainer](https://huggingface.co/docs/transformers/v4.16.1/en/main_classes/trainer#transformers.Trainer) class provides an API for feature-complete training in PyTorch for most standard use cases.

In our example we cannot use the `Trainer` out-of-the-box, since we need to pass in two models, the `Teacher` and the `Student` and compute the loss for both. But we can subclass the `Trainer` to create a `DistillationTrainer` which will take care of it and only overwrite the [compute_loss](https://github.com/huggingface/transformers/blob/c4ad38e5ac69e6d96116f39df789a2369dd33c21/src/transformers/trainer.py#L1962) method as well as the `init` method. In addition to this we also need to subclass the `TrainingArguments` to include the our distillation hyperparameters.


In [13]:
from transformers import TrainingArguments, Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F

class QuestionAnsweringTrainingArguments(TrainingArguments):
    def __init__(self, *args, max_length=384, doc_stride=128, version_2_with_negative=True, 
                 null_score_diff_threshold=0., n_best_size=20, max_answer_length=30,  alpha=.5, temperature=4, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.max_length = max_length
        self.doc_stride = doc_stride
        self.version_2_with_negative = version_2_with_negative
        self.null_score_diff_threshold = null_score_diff_threshold
        self.n_best_size = n_best_size
        self.max_answer_length = max_answer_length
        self.disable_tqdm = False
        self.alpha = alpha
        self.temperature = temperature
     

#export
class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.data_collator = default_data_collator
        self.compute_metrics = squad_metrics
        self.teacher = teacher_model
        # place teacher on same device as student
        self._move_model_to_device(self.teacher,self.model.device)
        self.teacher.eval()

    def compute_loss(self, model, inputs, return_outputs=False):
        # compute student output
        outputs_student = model(**inputs)
        student_loss=outputs_student.loss
        
        
        # compute teacher output
        with torch.no_grad():
          outputs_teacher = self.teacher(**inputs)
        assert outputs_student.start_logits.size() == outputs_teacher.start_logits.size()
        assert outputs_student.end_logits.size() == outputs_teacher.end_logits.size()
        # Soften probabilities and compute distillation loss
        loss_function = nn.KLDivLoss(reduction="batchmean")
        kl_loss = dict()
        kl_loss['start_logits'] = loss_function(
            F.log_softmax(outputs_student.start_logits / self.args.temperature, dim=-1),
            F.softmax(outputs_teacher.start_logits / self.args.temperature, dim=-1)) * (self.args.temperature ** 2)
        kl_loss['end_logits'] = loss_function(
            F.log_softmax(outputs_student.end_logits / self.args.temperature, dim=-1),
            F.softmax(outputs_teacher.end_logits / self.args.temperature, dim=-1)) * (self.args.temperature ** 2)
        
        loss = student_loss * self.args.alpha + ((kl_loss['start_logits'] + kl_loss["end_logits"]) / 2) * (1. - self.args.alpha)

        return (loss, outputs_student) if return_outputs else loss
        
    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        try:
            output = self.prediction_loop(
                eval_dataloader,
                description="Evaluation",
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
            )
        finally:
            self.compute_metrics = compute_metrics
        eval_dataset.set_format(type=eval_dataset.format["type"], columns=list(eval_dataset.features.keys()))

        if self.compute_metrics is not None:
            eval_preds = self._post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)
            # For some reason the eval_loss is not returned in output's metrics
            # Work around since NotebookProgressCallback assumes eval_loss key exists
            metrics['eval_loss'] = 'No log'

            self.log(metrics)
        else:
            metrics = {}
            
        for key in list(metrics.keys()):
            if not key.startswith(f"eval_"):
                metrics[f"eval_{key}"] = metrics.pop(key)

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, test_dataset, test_examples, ignore_keys=None):
        test_dataloader = self.get_test_dataloader(test_dataset)
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        try:
            output = self.prediction_loop(
                test_dataloader,
                description="Evaluation",
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
            )
        finally:
            self.compute_metrics = compute_metrics

        if self.compute_metrics is None:
            return output

        test_dataset.set_format(type=test_dataset.format["type"], columns=list(test_dataset.features.keys()))
        eval_preds = self._post_process_function(test_examples, test_dataset, output.predictions)
        metrics = self.compute_metrics(eval_preds)

        return PredictionOutput(predictions=eval_preds.predictions, label_ids=eval_preds.label_ids, metrics=metrics)
    
    
    def _post_process_function(self, examples, features, predictions):
        predictions = self._postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            version_2_with_negative=self.args.version_2_with_negative,
            n_best_size=self.args.n_best_size,
            max_answer_length=self.args.max_answer_length,
            null_score_diff_threshold=self.args.null_score_diff_threshold,
            output_dir=self.args.output_dir,
            is_world_process_zero=self.is_world_process_zero(),
        )
        if self.args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
        references = [{"id": ex["id"], "answers": ex['answers']} for ex in self.eval_examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)
    
    
    def _postprocess_qa_predictions(
        self,
        examples,
        features,
        predictions,
        version_2_with_negative= False,
        n_best_size = None,
        max_answer_length = None,
        null_score_diff_threshold = None,
        output_dir = None,
        prefix = None,
        is_world_process_zero = True,
    ):
        assert len(predictions) == 2, "`predictions` should be a tuple with two elements (start_logits, end_logits)."
        all_start_logits, all_end_logits = predictions
        assert len(predictions[0]) == len(features), f"Got {len(predictions[0])} predictions and {len(features)} features."

        example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
        features_per_example = collections.defaultdict(list)
        for i, feature in enumerate(features):
            features_per_example[example_id_to_index[feature["example_id"]]].append(i)

        all_predictions = collections.OrderedDict()

        for example_index, example in enumerate(tqdm(examples)):
            feature_indices = features_per_example[example_index]
            min_null_prediction = None
            prelim_predictions = []

            for feature_index in feature_indices:
                start_logits = all_start_logits[feature_index]
                end_logits = all_end_logits[feature_index]
                offset_mapping = features[feature_index]["offset_mapping"]
                token_is_max_context = features[feature_index].get("token_is_max_context", None)
                feature_null_score = start_logits[0] + end_logits[0]
                if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                    min_null_prediction = {
                        "offsets": (0, 0),
                        "score": feature_null_score,
                        "start_logit": start_logits[0],
                        "end_logit": end_logits[0],
                    }

                start_indexes = np.argsort(start_logits)[-1 : -self.args.n_best_size - 1 : -1].tolist()
                end_indexes = np.argsort(end_logits)[-1 : -self.args.n_best_size - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        if (
                            start_index >= len(offset_mapping)
                            or end_index >= len(offset_mapping)
                            or offset_mapping[start_index] is None
                            or offset_mapping[end_index] is None
                        ):
                            continue
                        if end_index < start_index or end_index - start_index + 1 > self.args.max_answer_length:
                            continue
                        if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                            continue
                        prelim_predictions.append(
                            {
                                "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                                "score": start_logits[start_index] + end_logits[end_index],
                                "start_logit": start_logits[start_index],
                                "end_logit": end_logits[end_index],
                            }
                        )
            if self.args.version_2_with_negative:
                prelim_predictions.append(min_null_prediction)
                null_score = min_null_prediction["score"]

            predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:self.args.n_best_size]
            if self.args.version_2_with_negative and not any(p["offsets"] == (0, 0) for p in predictions):
                predictions.append(min_null_prediction)

            context = example["context"]
            for pred in predictions:
                offsets = pred["offsets"]
                pred["text"] = context[offsets[0] : offsets[1]]

            if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
                predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

            scores = np.array([pred.pop("score") for pred in predictions])
            exp_scores = np.exp(scores - np.max(scores))
            probs = exp_scores / exp_scores.sum()

            for prob, pred in zip(probs, predictions):
                pred["probability"] = prob

            if not self.args.version_2_with_negative:
                all_predictions[example["id"]] = predictions[0]["text"]
            else:
                i = 0
                while predictions[i]["text"] == "":
                    i += 1
                best_non_null_pred = predictions[i]

                score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
                if score_diff > self.args.null_score_diff_threshold:
                    all_predictions[example["id"]] = ""
                else:
                    all_predictions[example["id"]] = best_non_null_pred["text"]
        return all_predictions

### Hyperparameter Definition, Model Loading

In [14]:
from huggingface_hub import HfFolder
import torch
from transformers import AutoModelForQuestionAnswering

def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(student_id)

args = QuestionAnsweringTrainingArguments(
    output_dir=repo_name,
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    fp16=True,
    learning_rate=6e-5,
    seed=33,
    # logging & evaluation strategies
    logging_dir=f"{repo_name}/logs",
    logging_strategy="epoch", # to get more information to TB
    evaluation_strategy='epoch',
    save_strategy="epoch",
    report_to="tensorboard",
    # push to hub parameters
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repo_name,
    hub_token=HfFolder.get_token(),
    # distilation parameters
    alpha=0.5,
    temperature=4.0
)

# define model
teacher_model = AutoModelForQuestionAnswering.from_pretrained(
    teacher_id,
)

trainer = QuestionAnsweringTrainer(
    args=args,
    model_init=model_init,
    teacher_model=teacher_model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    eval_examples=eval_examples,
    tokenizer=tokenizer
)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_output

In [15]:
trainer.train()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.weight', 'qa_output

  0%|          | 0/78 [00:00<?, ?it/s]

{'loss': 4.4486, 'learning_rate': 4.0769230769230773e-05, 'epoch': 1.0}


c:\Users\Kasra Sohrab\.conda\envs\nlp\Lib\site-packages\transformers\trainer_pt_utils.py:411: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

...

c:\Users\Kasra Sohrab\.conda\envs\nlp\Lib\site-packages\transformers\trainer_pt_utils.py:411: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

Trainer is attempting to log a value of "No log" of type <class 'str'> for key "eval/loss" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


[{'id': '5733ea04d058e614000b6595', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad26a5fd7d075001a42931b', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a664447c2b11c001a425eef', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad3ff1b604f3c001a3ffc74', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad25d39d7d075001a428eda', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad265d2d7d075001a4291c6', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a1c8ea7b4fb5d00187146d1', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a57c667770dc0001aeefd69', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5705fd8475f01819005e7841', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5726431d271a42140099d7f9', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a8929e43b2508001a72a4de', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5706143575f0

### Evaluation metric

we can create a `compute_metrics` function to evaluate our model on the test set. This function will be used during the training process to compute the `accuracy` & `f1` of our model.

In [ ]:
from datasets import load_metric
import numpy as np

def count_parameters(model):
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    trainable_pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return pytorch_total_params

print(count_parameters(teacher_model))
print(count_parameters(trainer.model))

c:\Users\Kasra Sohrab\.conda\envs\nlp\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\Kasra Sohrab\.conda\envs\nlp\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to loa

334094338
13483522


In [110]:
trainer.evaluate()

c:\Users\Kasra Sohrab\.conda\envs\nlp\Lib\site-packages\transformers\trainer_pt_utils.py:411: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

<function squad_metrics at 0x000001D8B0AC14E0>


  0%|          | 0/320 [00:00<?, ?it/s]

[{'id': '5733ea04d058e614000b6595', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad26a5fd7d075001a42931b', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a664447c2b11c001a425eef', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad3ff1b604f3c001a3ffc74', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad25d39d7d075001a428eda', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5ad265d2d7d075001a4291c6', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a1c8ea7b4fb5d00187146d1', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a57c667770dc0001aeefd69', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5705fd8475f01819005e7841', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5726431d271a42140099d7f9', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5a8929e43b2508001a72a4de', 'prediction_text': '', 'no_answer_probability': 0.0}, {'id': '5706143575f0

Trainer is attempting to log a value of "No log" of type <class 'str'> for key "eval/loss" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 'No log',
 'eval_exact': 52.1875,
 'eval_f1': 52.1875,
 'eval_total': 320,
 'eval_HasAns_exact': 0.0,
 'eval_HasAns_f1': 0.0,
 'eval_HasAns_total': 150,
 'eval_NoAns_exact': 98.23529411764706,
 'eval_NoAns_f1': 98.23529411764706,
 'eval_NoAns_total': 170,
 'eval_best_exact': 53.125,
 'eval_best_exact_thresh': 0.0,
 'eval_best_f1': 53.125,
 'eval_best_f1_thresh': 0.0}

## Training

Start training with calling `trainer.train`

In [14]:
"""
trainer = DistillationTrainer(
    student_model,
    training_args,
    teacher_model=teacher_model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
"""

'\ntrainer = DistillationTrainer(\n    student_model,\n    training_args,\n    teacher_model=teacher_model,\n    train_dataset=tokenized_datasets["train"],\n    eval_dataset=tokenized_datasets["validation"],\n    data_collator=data_collator,\n    tokenizer=tokenizer,\n    compute_metrics=compute_metrics,\n)\n'

start training using the `DistillationTrainer`.

## Hyperparameter Search for Distillation parameter `alpha` & `temperature` with optuna

The parameter `alpha` & `temparature` in the `DistillationTrainer` can also be used when doing Hyperparamter search to maxizime our "knowledge extraction". As Hyperparamter Optimization framework are we using [Optuna](https://optuna.org/), which has a integration into the `Trainer-API`. Since we the `DistillationTrainer` is a sublcass of the `Trainer` we can use the `hyperparameter_search` without any code changes.


In [16]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


To do Hyperparameter Optimization using `optuna` we need to define our hyperparameter space. In this example we are trying to optimize/maximize the `num_train_epochs`, `learning_rate`, `alpha` & `temperature` for our `student_model`.

In [17]:
def hp_space(trial):
    return {
      "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 10),
      "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3 ,log=True),
      "alpha": trial.suggest_float("alpha", 0, 1),
      "temperature": trial.suggest_int("temperature", 2, 30),
      }

To start our Hyperparmeter search we just need to call `hyperparameter_search` provide our `hp_space` and number of trials to run.

In [18]:
def student_init():
    return AutoModelForQuestionAnswering.from_pretrained(
        student_id,
    )

trainer = DistillationTrainer(
    model_init=student_init,
    args=training_args,
    teacher_model=teacher_model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
best_run = trainer.hyperparameter_search(
    n_trials=50,
    direction="maximize",
    hp_space=hp_space
)

print(best_run)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-04-19 01:25:51,439] A new study created in memory with name: no-name-11c6ea15-9913-4e54-8f02-6ebaaab6fbbb
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1581 [00:00<?, ?it/s]

{'loss': 1.4461, 'grad_norm': 32.83808898925781, 'learning_rate': 5.9109468466799715e-05, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.8842347860336304, 'eval_accuracy': 0.9151376146788991, 'eval_runtime': 1.4618, 'eval_samples_per_second': 596.511, 'eval_steps_per_second': 4.789, 'epoch': 1.0}
{'loss': 0.7, 'grad_norm': 52.87685775756836, 'learning_rate': 2.96107091088419e-05, 'epoch': 2.0}


...

{'eval_loss': 0.3680877685546875, 'eval_accuracy': 0.9174311926605505, 'eval_runtime': 1.4488, 'eval_samples_per_second': 601.891, 'eval_steps_per_second': 4.832, 'epoch': 4.0}
{'loss': 0.1029, 'grad_norm': 4.315232753753662, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.37717682123184204, 'eval_accuracy': 0.9174311926605505, 'eval_runtime': 1.4394, 'eval_samples_per_second': 605.794, 'eval_steps_per_second': 4.863, 'epoch': 5.0}
{'train_runtime': 567.0988, 'train_samples_per_second': 593.803, 'train_steps_per_second': 4.646, 'train_loss': 0.19365764668589525, 'epoch': 5.0}


[I 2024-04-19 06:15:03,963] Trial 49 finished with value: 0.9174311926605505 and parameters: {'num_train_epochs': 5, 'learning_rate': 6.19096424666421e-05, 'alpha': 0.982936133975336, 'temperature': 22}. Best is trial 13 with value: 0.9220183486238532.


BestRun(run_id='13', objective=0.9220183486238532, hyperparameters={'num_train_epochs': 9, 'learning_rate': 4.854751380955516e-05, 'alpha': 0.8585651118406669, 'temperature': 23}, run_summary=None)


Since optuna is just finding the best hyperparameters we need to fine-tune our model again using the best hyperparamters from the `best_run`.

In [19]:
# overwrite initial hyperparameters with from the best_run
for k,v in best_run.hyperparameters.items():
    setattr(training_args, k, v)

# Define a new repository to store our distilled model
best_model_ckpt = "electra-distilled-best-sst"
training_args.output_dir = best_model_ckpt

# Data:

In [20]:
print(count_parameters(student_model))

13549314


We have overwritten the default Hyperparameters with the one from our `best_run` and can start the training now.

In [21]:
# Create a new Trainer with optimal parameters
optimal_trainer = DistillationTrainer(
    student_model,
    training_args,
    teacher_model=teacher_model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

optimal_trainer.train()


# save best model, metrics and create model card
trainer.create_model_card(model_name=training_args.hub_model_id)
trainer.push_to_hub()

  0%|          | 0/4743 [00:00<?, ?it/s]

{'loss': 0.2625, 'grad_norm': 11.34455394744873, 'learning_rate': 4.3163581221778226e-05, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.5787972807884216, 'eval_accuracy': 0.911697247706422, 'eval_runtime': 1.4545, 'eval_samples_per_second': 599.512, 'eval_steps_per_second': 4.813, 'epoch': 1.0}
{'loss': 0.214, 'grad_norm': 29.352754592895508, 'learning_rate': 3.7769413020716535e-05, 'epoch': 2.0}


  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.5945138335227966, 'eval_accuracy': 0.9220183486238532, 'eval_runtime': 1.448, 'eval_samples_per_second': 602.228, 'eval_steps_per_second': 4.834, 'epoch': 8.0}
{'loss': 0.0889, 'grad_norm': 14.889159202575684, 'learning_rate': 3.070683985424109e-08, 'epoch': 9.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.6007155776023865, 'eval_accuracy': 0.9220183486238532, 'eval_runtime': 1.4604, 'eval_samples_per_second': 597.088, 'eval_steps_per_second': 4.793, 'epoch': 9.0}
{'train_runtime': 1017.9606, 'train_samples_per_second': 595.446, 'train_steps_per_second': 4.659, 'train_loss': 0.1510109045875995, 'epoch': 9.0}


CommitInfo(commit_url='https://huggingface.co/kasohrab/electra-distilled-sst/commit/593f17c76d2ba59af9f3f6046449f7c743fef377', commit_message='End of training', commit_description='', oid='593f17c76d2ba59af9f3f6046449f7c743fef377', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from huggingface_hub import HfApi

whoami = HfApi().whoami()
username = whoami['name']

print(f"https://huggingface.co/{username}/{repo_name}")

https://huggingface.co/kasohrab/electra-distilled-sst


## Results

We were able to achieve a `accuracy` of 0.8337, which is a very good result for our model. Our distilled `Tiny-Bert` has 96% less parameters than the teacher `bert-base` and runs ~46.5x faster while preserving over 90% of BERT’s performances as measured on the SST2 dataset.

| model | Parameter | Speed-up | Accuracy |
|------------|-----------|----------|----------|
| BERT-base  | 109M      | 1x       | 93%      |
| tiny-BERT  | 4M        | 46.5x    | 83%      |

_Note: The [FastFormers paper](https://arxiv.org/abs/2010.13382) uncovered that the biggest boost in performance is observerd when having 6 or more layers in the student. The [google/bert_uncased_L-2_H-128_A-2](https://huggingface.co/google/bert_uncased_L-2_H-128_A-2) we used only had 2, which means when changing our student to, e.g. `distilbert-base-uncased` we should better performance in terms of accuracy._

If you are now planning to implement and add task-specific knowledge distillation to your models. I suggest to take a look at the [sagemaker-distillation](https://github.com/philschmid/knowledge-distillation-transformers-pytorch-sagemaker/blob/master/sagemaker-distillation.ipynb), which shows how to run task-specific knowledge distillation on Amazon SageMaker. For the example i created a script deriving this notebook to make it as easy as possible to use for you. You only need to define your `teacher_id`, `student_id` as well as your `dataset` config to run task-specific knowledge distillation for `text-classification`.

```python
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'teacher_id':'textattack/bert-base-uncased-SST-2',           
    'student_id':'google/bert_uncased_L-2_H-128_A-2',           
    'dataset_id':'glue',           
    'dataset_config':'sst2',             
    # distillation parameter
    'alpha': 0.5,
    'temparature': 4,
    # hpo parameter
    "run_hpo": True,
    "n_trials": 100,            
}

# create the Estimator
huggingface_estimator = HuggingFace(..., hyperparameters=hyperparameters)

# start knwonledge distillation training
huggingface_estimator.fit()
```